# Практическая часть 1
Исследование табличных данных и простые «потоки» обработки в Python / Google Colab

## Блок 0. Подготовка окружения

Импортировать необходимые библиотеки для работы с таблицами, визуализацией и моделями.
Настроить отображение таблиц.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

%matplotlib inline

pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 120)

## Блок 1. Набор данных Iris: загрузка и первичный анализ

### 1.1. Загрузка набора данных Iris

Загрузить встроенный набор данных Iris, сформировать DataFrame и добавить человекочитаемое название вида.

In [ ]:
iris = datasets.load_iris(as_frame=True)
df_iris = iris.frame.copy()
df_iris["species"] = df_iris["target"].map(dict(enumerate(iris.target_names)))
df_iris.head()

### 1.2. Общая информация о данных

Получить размерность датасета, типы данных по столбцам и количество пропусков.

In [ ]:
print("Форма датасета (строки, столбцы):", df_iris.shape)
print()
print("Типы данных:")
print(df_iris.dtypes)
print()
print("Пропуски по столбцам:")
print(df_iris.isna().sum())

### 1.3. Описательная статистика

Вывести базовые статистики по числовым признакам (минимум, максимум, среднее, квартили).

In [ ]:
df_iris.describe()

### 1.4. Распределения признаков

Построить гистограммы распределения числовых признаков.

In [ ]:
numeric_cols = iris.feature_names
df_iris[numeric_cols].hist(figsize=(10, 8))
plt.suptitle("Распределение числовых признаков Iris", y=1.02)
plt.tight_layout()
plt.show()

### 1.5. Диаграмма рассеяния

Построить диаграмму рассеяния для длины и ширины лепестка с раскраской по виду растения.

In [ ]:
plt.figure(figsize=(6, 5))
scatter = plt.scatter(
    df_iris["petal length (cm)"],
    df_iris["petal width (cm)"],
    c=df_iris["target"],
    alpha=0.8
)
plt.xlabel("Petal length (cm)")
plt.ylabel("Petal width (cm)")
plt.title("Iris: диаграмма рассеяния по лепесткам")
handles, _ = scatter.legend_elements()
plt.legend(handles, iris.target_names, title="Species", loc="lower right")
plt.show()

## Блок 2. Простой «поток данных» над таблицей

### 2.1. Последовательность преобразований

Построить простой поток обработки данных:
1. Выбрать только часть столбцов.
2. Отфильтровать строки по условию.
3. Добавить новый признак как функцию от существующих.

In [ ]:
df_flow = df_iris[["sepal length (cm)", "sepal width (cm)", "species"]].copy()
df_flow = df_flow[df_flow["sepal length (cm)"] > 5.0]
df_flow["sepal_ratio"] = df_flow["sepal length (cm)"] / df_flow["sepal width (cm)"]
df_flow.head()

### 2.2. Визуализация результата потока

Визуализировать результат после всех шагов потока: диаграмма рассеяния для длины и ширины чашелистиков, раскрашенная по видам.

In [ ]:
plt.figure(figsize=(6, 5))
for species_name, sub_df in df_flow.groupby("species"):
    plt.scatter(
        sub_df["sepal length (cm)"],
        sub_df["sepal width (cm)"],
        label=species_name,
        alpha=0.8
    )
plt.xlabel("Sepal length (cm)")
plt.ylabel("Sepal width (cm)")
plt.title("Фильтрованные и преобразованные признаки")
plt.legend()
plt.show()

## Блок 3. «Виджеты и каналы» через sklearn.Pipeline

### 3.1. Подготовка выборок

Подготовить матрицу признаков и целевую переменную.
Разбить данные на обучающую и тестовую выборки.

In [ ]:
X = df_iris[iris.feature_names]
y = df_iris["species"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

### 3.2. Построение и обучение Pipeline

Построить конвейер из трёх шагов:
1. Масштабирование признаков.
2. PCA до двух компонент.
3. Логистическая регрессия.

Обучить конвейер и оценить точность на тестовой выборке.

In [ ]:
pipe = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=2)),
        ("clf", LogisticRegression(max_iter=1000))
    ]
)

pipe.fit(X_train, y_train)
print("Точность на тестовой выборке:", pipe.score(X_test, y_test))

### 3.3. Получение 2D-представления после PCA

Получить представление объектов в двумерном пространстве главных компонент и собрать его в отдельный DataFrame.

In [ ]:
X_pca = pipe.named_steps["pca"].transform(
    pipe.named_steps["scaler"].transform(X)
)

df_pca = pd.DataFrame(
    X_pca,
    columns=["PC1", "PC2"]
)
df_pca["species"] = y.values
df_pca.head()

### 3.4. Визуализация в пространстве главных компонент

Построить диаграмму рассеяния в координатах PC1–PC2, раскрашенную по видам.

In [ ]:
plt.figure(figsize=(6, 5))
for species_name, sub_df in df_pca.groupby("species"):
    plt.scatter(
        sub_df["PC1"],
        sub_df["PC2"],
        label=species_name,
        alpha=0.8
    )
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Iris после StandardScaler + PCA")
plt.legend()
plt.show()

## Блок 4. Загрузка собственных данных из CSV

### 4.1. Формирование собственного набора данных

Сформировать небольшой набор данных о людях (имя, пол, рост, вес, цвет глаз и волос) и представить его в виде DataFrame.

In [ ]:
people_data = {
    "name": ["Anna", "Boris", "Clara", "Dmitry", "Elena", "Fedor"],
    "gender": ["female", "male", "female", "male", "female", "male"],
    "height_cm": [165, 180, 170, 175, 160, 182],
    "weight_kg": [55, 78, 60, 72, 52, 85],
    "eye_color": ["blue", "brown", "green", "brown", "blue", "brown"],
    "hair_color": ["blond", "black", "brown", "brown", "red", "black"],
}

df_people = pd.DataFrame(people_data)
df_people

### 4.2. Сохранение в CSV

Сохранить таблицу в CSV-файл, имитируя подготовку собственного набора данных.

In [ ]:
csv_path = "people_physical_data.csv"
df_people.to_csv(csv_path, index=False)
csv_path

### 4.3. Загрузка данных из CSV

Загрузить созданный CSV-файл в новый DataFrame и проверить структуру данных.

In [ ]:
df_people_loaded = pd.read_csv(csv_path)
df_people_loaded

### 4.4. Проверка структуры и выделение признаков/цели

Вывести список столбцов и типы данных.
Разделить таблицу на матрицу признаков и целевую переменную (пол), подготовив данные к последующим практикам.

In [ ]:
print("Столбцы:", list(df_people_loaded.columns))
print()
print("Типы данных:")
print(df_people_loaded.dtypes)

In [ ]:
X_people = df_people_loaded[["height_cm", "weight_kg", "eye_color", "hair_color"]]
y_people = df_people_loaded["gender"]
X_people.head(), y_people.head()